In [1]:
import tensorflow as tf
import numpy as np
import dbGeneration as db
import random

# Data Pre-Processing

In [2]:
tensorInputs, raw_labels = db.get_tensorinputs_and_labels()
tensorInputs = np.asarray(tensorInputs)


Returned data points in 81 batches


In [3]:
n = 0


for i in tensorInputs:
    n += len(i)
    
print(n, "training samples")

225447 training samples


In [4]:
print(len(raw_labels))

81


In [5]:
print(np.asarray(tensorInputs.shape))

tensorInputs2 = []
raw_labels2 = []

for i in tensorInputs:
    for j in i:
        tensorInputs2.append(j)

for i in raw_labels:
    for j in i:
        raw_labels2.append(j)

print(len(tensorInputs2))
print(len(raw_labels2))

print(tensorInputs2[0])
print(raw_labels2[0])

[81]
225447
225447
[[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0. -1.  1.  0.  1.  0.  0.  2.]
 [ 2.  0.  0. -1. -1.  1. -1.  0.  0.  2.]
 [ 2.  0.  0. -1. -1. -1. -1.  0.  0.  2.]
 [ 2.  0.  0.  0.  1.  1.  1.  1.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]]
-1


In [6]:
raw_labels = raw_labels2
tensorInputs = tensorInputs2
del(raw_labels2)
del(tensorInputs2)

In [7]:
print(raw_labels[:10])

[-1, -1, 1, 1, 1, 1, 1, 1, 1, -1]


### Convert labels to one-hot vector

In [8]:
pre_labels = []
for i in raw_labels:
    if i == -1:
        pre_labels.append([1,0])
    else:
        pre_labels.append([0,1])

pre_labels = np.asarray(pre_labels)

In [9]:
print(pre_labels.shape)
print(pre_labels)

(225447, 2)
[[1 0]
 [1 0]
 [0 1]
 ..., 
 [0 1]
 [1 0]
 [1 0]]


### Define training and testing data

In [10]:
print(len(tensorInputs))

225447


In [11]:
BATCH_SIZE = 64
N_SAMPLES = len(tensorInputs)
N_TRAIN = 220000
N_VALIDATION = int((N_SAMPLES - N_TRAIN)/2)
N_BATCHES = int(N_TRAIN/BATCH_SIZE)

In [12]:
print(N_BATCHES)

3437


## Shuffle data to avoid grouping among games

In [13]:
def get_randomised_batches(training_data, training_labels):
    pairs = []
    for i in range(len(training_data)):
        pairs.append([training_data[i], training_labels[i]])
        
    random.shuffle(pairs)
    x = []
    y = []

    for i in pairs:
        x.append(np.reshape(i[0], (100)))
        y.append(i[1])
        
    return x, y

In [14]:
train_gameboards = tensorInputs[:N_TRAIN]
validation_gameboards = tensorInputs[N_TRAIN:N_TRAIN+N_VALIDATION]
test_gameboards = tensorInputs[N_TRAIN+N_VALIDATION:]

train_labels = pre_labels[:N_TRAIN]
validation_labels = pre_labels[N_TRAIN:N_TRAIN+N_VALIDATION]
test_labels = pre_labels[N_TRAIN+N_VALIDATION:]

In [15]:
print(len(train_gameboards))
print(len(validation_gameboards))
print(len(test_gameboards))

220000
2723
2724


In [16]:
print(test_gameboards[0], test_labels[0])

[[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  1.  1.  1.  0.  0.  2.]
 [ 2.  0.  0. -1.  1.  1. -1. -1.  0.  2.]
 [ 2.  0.  0. -1.  1. -1.  1. -1.  0.  2.]
 [ 2.  0.  0. -1.  1.  1.  1.  1.  0.  2.]
 [ 2.  0.  0.  0. -1. -1.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0. -1.  0.  0.  0.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]] [1 0]


# Model Definition

In [17]:
gameboards = tf.placeholder(tf.float32, (None, 100), name="gameBoards")

gameboards2d = tf.reshape(gameboards, (-1, 10, 10, 1), name="gameBoards2d")



In [18]:
# First Convulutional layer, small 2x2 filter
conv1 = tf.layers.conv2d(gameboards2d, 64, 2, padding="same", name="Conv1", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv2d(pool1, 128, 3, padding="same", name="Conv2", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

conv3 = tf.layers.conv2d(pool2, 128, 4, padding="same", name="Conv3", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(conv3, 2, 2, name="Pool3")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
# Flatten out the pooling - GET THIS NUMBER FROM POOLx.SHAPE
pool2_flat = tf.reshape(pool3, (-1, int(1*1*128)), name="Pool2_Flat")


# The dropout allows us to train a subset of the neurons at any given iteration.  
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")


# A dense layer with dropout
# DENSE - a fully connected linear transofmration of every dimension of the data
dense = tf.layers.dense(pool2_flat, int(128), activation=tf.nn.relu, name="Dense")

# DROPOUT - if set to 0.5, rendomly select 50% of the neurons to ignore (different with each computation)
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

dense2 = tf.layers.dense(dropout, int(128), activation=tf.nn.relu, name="Dense2")
dropout2 = tf.nn.dropout(dense2, keep_prob, name="Dropout2")

"""dense3 = tf.layers.dense(dropout, int(128), activation=tf.nn.relu, name="Dense3")
dropout3 = tf.nn.dropout(dense3, keep_prob, name="Dropout3")"""

# A dense layer to classify the final values. Only 2 neurons. 
predictions = tf.layers.dense(dropout2, 2, activation=None, name="Predictions")

In [19]:
pool3.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(128)])

In [20]:
# None, as in, this is not yet defined, there could be any number of them input. 
# 2, as in, there are two elements in the one-hot vector

labels = tf.placeholder(tf.int32, [None, 2], name="labels")

In [21]:
# This loss is the elementwise loss
#loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

with tf.name_scope("Loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=predictions))

"""with tf.name_scope("Loss"):
    loss = tf.losses.mean_squared_error(labels=labels, predictions=predictions)"""

with tf.name_scope("Optimizer"):
    train = tf.train.AdamOptimizer(learning_rate=0.001, name="Adam").minimize(loss)

In [22]:
with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(labels, 1), tf.argmax(predictions, 1)), tf.float32), name="Mean")

## Define an error rate to evaluate model performance


In [23]:
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)

<tf.Tensor 'Error_1:0' shape=() dtype=string>

In [24]:
number_of_batches = int(N_TRAIN / BATCH_SIZE)
print(number_of_batches)

3437


In [25]:
sess = tf.Session()

In [26]:
sess.run(tf.global_variables_initializer())

In [27]:
for epoch in range(1):
    #randomise the data
    gb, lbl  = get_randomised_batches(train_gameboards, train_labels)
    
    for i in range(N_BATCHES):
        first_index = i*BATCH_SIZE
        second_index = (i*BATCH_SIZE) + BATCH_SIZE
        batch = gb[first_index:second_index]
        batchLabels = lbl[first_index:second_index]

        #print(np.asarray(batch).shape)

        if i % 100 == 0:
            Error, Loss = sess.run([error, loss], feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})
            print("{}, Loss: {}, \tError: {}%".format(i, Loss, Error*100))
        sess.run(train, feed_dict={gameboards:batch, labels:batchLabels, keep_prob:0.5})


0, Loss: 0.6959189176559448, 	Error: 51.5625%
100, Loss: 0.6776396632194519, 	Error: 42.1875%
200, Loss: 0.7022673487663269, 	Error: 51.5625%
300, Loss: 0.6592915058135986, 	Error: 39.0625%
400, Loss: 0.6234550476074219, 	Error: 39.0625%
500, Loss: 0.6642321348190308, 	Error: 51.5625%
600, Loss: 0.6565261483192444, 	Error: 37.5%
700, Loss: 0.6397536396980286, 	Error: 42.1875%
800, Loss: 0.6840679049491882, 	Error: 45.3125%
900, Loss: 0.6022106409072876, 	Error: 35.9375%
1000, Loss: 0.637906551361084, 	Error: 37.5%
1100, Loss: 0.6105183959007263, 	Error: 26.5625%
1200, Loss: 0.6613262891769409, 	Error: 43.75%
1300, Loss: 0.6630628108978271, 	Error: 40.625%
1400, Loss: 0.6197479367256165, 	Error: 32.8125%
1500, Loss: 0.6690207719802856, 	Error: 40.625%
1600, Loss: 0.7126659750938416, 	Error: 45.3125%
1700, Loss: 0.64258873462677, 	Error: 32.8125%
1800, Loss: 0.7092994451522827, 	Error: 51.5625%
1900, Loss: 0.616108775138855, 	Error: 37.5%
2000, Loss: 0.6621047854423523, 	Error: 40.625%
2

In [28]:
val = []

for i in validation_gameboards:
    val.append(np.reshape(i, (100)))

In [29]:
#test = []

#for i in test_gameboards:
    #test.append(np.reshape(i, (100)))

In [30]:
Error = sess.run(error, feed_dict={gameboards:val, labels:validation_labels, keep_prob:1.0})
print(Error)

0.403966


In [31]:
#Error = sess.run(error, feed_dict={gameboards:test, labels:test_labels, keep_prob:1.0})
#print(Error)

In [32]:
saver = tf.train.Saver()
saver.save(sess, "nn_model\\")

'nn_model\\'

In [48]:
for i in range(100):
    r = sess.run(predictions, feed_dict={gameboards:[val[i]], keep_prob:1.0})
    print(r, validation_labels[i])

[[ 0.16845234 -0.1164927 ]] [1 0]
[[ 0.0939784  -0.05144904]] [1 0]
[[ 0.24084786 -0.14548191]] [1 0]
[[ 0.68123651 -0.33546242]] [1 0]
[[ 0.13589826 -0.10234954]] [0 1]
[[ 0.10060694 -0.03405943]] [0 1]
[[-0.03575656  0.21318907]] [0 1]
[[-0.05135381  0.22725052]] [1 0]
[[ 0.06522098 -0.03346411]] [1 0]
[[ 0.66462612 -0.28097069]] [1 0]
[[ 0.03202344  0.02583835]] [0 1]
[[-0.04063651  0.19388989]] [0 1]
[[-0.35128403  0.80436122]] [0 1]
[[-0.32565069  0.80916274]] [0 1]
[[ 0.08136741 -0.05525216]] [1 0]
[[ 4.92718554 -2.24777603]] [1 0]
[[ 0.11098722 -0.08823634]] [1 0]
[[ 0.12697744 -0.11159419]] [1 0]
[[ 0.21536201 -0.13632891]] [1 0]
[[ 0.20691366 -0.13188654]] [1 0]
[[ 0.05483514 -0.01432633]] [0 1]
[[ 0.03993898 -0.01252807]] [0 1]
[[ 0.15263148 -0.11417112]] [0 1]
[[ 0.34420252 -0.17891502]] [0 1]
[[ 0.10886383 -0.08881157]] [0 1]
[[ 0.48385161 -0.21890791]] [0 1]
[[ 0.13496089 -0.10158335]] [1 0]
[[ 0.12445832 -0.10056013]] [1 0]
[[ 0.18941447 -0.12931795]] [1 0]
[[ 0.23126787 

In [35]:
print(r)

[[ 0.16845234 -0.1164927 ]]
